# Temporal Network Visualisation

The `pathpy` package provides special support for the analysis of temporal networks data via its `TemporalNetwork` class. It is suitable for data that captures time-stamped edges  $(v,w,t)$  instantaneously occurring at discrete time stamps $t$. Let us start by creating an empty instance of this class.

In [ ]:
mport pathpy as pp
import numpy as np
import sqlite3

import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from collections import defaultdict

plt.style.use('default')
sns.set_style("whitegrid")

In [ ]:
t = pp.TemporalNetwork()
print(t)

The TemporalNetwork instance $t$ stores two key information: a list of nodes t.nodes and a collection t.tedges of time-stamped edges $(v,w,t)$. Let us add some time-stamped edges to this instance.

In [ ]:
t.add_edge('a', 'b', 1)
t.add_edge('b', 'a', 3)
t.add_edge('b', 'c', 3)
t.add_edge('d', 'c', 4)
t.add_edge('c', 'd', 5)
t.add_edge('c', 'b', 6)
print(t)

We get basic summary statistics, such as the number of time-stamped interactions, the minimum and maximum timestamp, the duration of the observation, the number of different timestamps, as well as the average, minimum, and maximum time difference between consecutive time-stamped edges.

Above we used integer timestamps, which represent discrete time units. But we often have data where time is given in terms of a date and/or time of day. Luckily, pathpy supports arbitrary timestamp formats. Let us try this in an example.

In [ ]:
t_realtime = pp.TemporalNetwork()
t_realtime.add_edge('a', 'b', '2018-08-22 09:30:22')
t_realtime.add_edge('b', 'c', '2018-08-22 09:30:25')
t_realtime.add_edge('c', 'a', '2018-08-22 10:30:25')
print(t_realtime)

for e in t_realtime.tedges:
    print(e)

We observe that `pathpy` internally converts such timestamps into UNIX timestamps. For custom formats, we can set a custom `timestamp_format` parameter that will be used for this conversion. After the conversion, all time units will be in seconds (see e.g. the min/max inter-event time).

Just like other pathpy objects, we can directly embed interactive visualisations of a TemporalNetwork in-line in jupyter. Let us try this with our first example instance $t$.

In [ ]:
t

Using the default parameters, this visualisation is too fast. Luckily, we can use the generic `pp.visualisation.plot` function to pass a style for the visualisation. We can use all parameters that we used for static networks, plus additional parameters that influence temporal aspects of the visualisation.

Of particular importance are the parameters `ms_per_frame` and `ts_per_frame`: The first specifies how many time units will be shown in one frame of the visualisation, allowing us to compress the visualisation by showing multiple timestamps in a single frame. This is helpful when you want to coarse-grain visualisations of high-resolution temporal network data. The parameter `ms_per_frame` defines the target frame rate of the visualisation by adjusting how many milliseconds each frame is displayed.

Two more parameters will influence the force-directed layout algorithm, that is used to position nodes in the network. In a temporal network, the question is which time-stamped edges should be taken into account for the force-calculation at any given time stamp. If we only consider currently active edges, the layout will change too fast to recognize interesting patterns. If we consider all edges at every time step, node positions will be static despite the dynamics of edges. In real settings we want a compromise between those extremes, i.e. we specify a time window around the current time stamp within which edges are taken into account in the force-directed layout calculation. We can achieve this by setting the number of timestamps to consider before and after the currently shown frame via the `look_ahead` and `look_behind` parameters.

Finally, we can style active and inactive nodes and edges individually via the parameters `active_edge_width`, `inactive_edge_width`, `active_node_color`, and `inactive_node_color`. This allows us to change the color and/or size of nodes/edges whenever they are involved in an interaction.

In [ ]:
style = {    
  'ts_per_frame': 1, 
  'ms_per_frame': 2000,
  'look_ahead': 2, 
  'look_behind': 2, 
  'node_size': 15, 
  'inactive_edge_width': 2,
  'active_edge_width': 4, 
  'label_color' : '#ffffff',
  'd3js_path': 'http://localhost:8888/notebooks/d3.v4.min.js',
  'label_size' : '24px',
  'label_offset': [0,5]
  }
pp.visualisation.plot(t, **style)

This generates an embedded interactive visualisation, i.e. you can pan and zoom, or drag nodes. The controls in the top part of the visualisation allow you to stop, start or restart the simulation.

We can easily save such interactive visualisations as stand-alone HTML5 files, which can be distributed via the Web.

In [ ]:
pp.visualisation.export_html(t, '01_temporal_network.html', **style)

template based visualization?